In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.insert(0, "../")

In [3]:
from src.model.deeplabv3 import TrashDetector
from src.data.dataset import TrashDataset
from torch.utils.data import DataLoader
from pytorch_lightning.loggers import TensorBoardLogger

In [4]:
from torchvision import transforms
import torch
import pytorch_lightning as pl
from pathlib import Path

In [5]:
val_size = 0.2
batch_size = 2
CONFIG = {
    "img_size": 224,
    "batch_size": batch_size,
    "val_size": val_size,
}

In [6]:
# images = list(Path("../data/raw/train").glob("*.jpg"))
# videos = {img.name[4:8] for img in images}
# len(videos) * 2

# DataLoader

In [7]:
transform = preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((CONFIG["img_size"], CONFIG["img_size"])),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [8]:
dataset = TrashDataset("../data/raw/train-all.csv", "../data/raw/train", transform=transform)

In [9]:
train_set, val_set = torch.utils.data.random_split(
    dataset, 
    [
        len(dataset) - int(len(dataset) * val_size) , 
        int(len(dataset) * val_size) 
    ]
)

In [10]:
train_dataloader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=8)
val_dataloader = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=8)

# Model

In [11]:
model = TrashDetector()

In [12]:
logger = TensorBoardLogger("../logs", name="DeepLabV3")
trainer = pl.Trainer(logger=logger)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [13]:
trainer.fit(model, train_dataloader, val_dataloader)


  | Name  | Type      | Params
------------------------------------
0 | model | DeepLabV3 | 61.0 M
------------------------------------
61.0 M    Trainable params
0         Non-trainable params
61.0 M    Total params
243.944   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/home/renesat/0. PARA/1. Projects/Ecodrone/NN/.venv/lib/python3.9/site-packages/torch/_tensor.py:493: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


mIoU = 0.0
mDSC = mDSC=0.0


Training: -1it [00:00, ?it/s]

/home/renesat/0. PARA/1. Projects/Ecodrone/NN/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1051: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [14]:
model

TrashDetector(
  (model): DeepLabV3(
    (backbone): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       

In [15]:
pl.__version__


'1.4.9'